In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import tqdm
import zipfile
import torch
import pandas as pd


In [2]:
fhlo = pd.read_csv("../../input_data/fully_hand_labelled_output.csv")

In [3]:
fhlo

,Unnamed: 0.1,Unnamed: 0,label_id,text,Roken,Alcohol,Drugs
0,0,0,1.0,Beloop: \tPatiÃ«nte heeft 10 minuten van te vo...,Niets gevonden,Niets gevonden,Niets gevonden
1,1,1,1.0,Beloop: \tG5P4 36 wkGrav 1e lijnALL geen Hb Pa...,Niets gevonden,Niets gevonden,Niets gevonden
2,2,2,1.0,Conclusie: \tNormale nacontrole. Kijkt goed te...,Niets gevonden,Niets gevonden,Niets gevonden
3,3,3,1.0,Beleid: \tAlgemeen: Dagopname voor 3x PC a 2....,Niets gevonden,Niets gevonden,Niets gevonden
4,4,4,1.0,Anamnese: \t34W4D,Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...,...,...
4695,4695,95,2.0,Beloop: \tCONSULTENKAMERGezien door co-ass Y. ...,Geen gebruiker,Geen gebruiker,Geen gebruiker
4696,4696,96,2.0,Reden van komst / Verwijzing: \tReden verwijzi...,Geen gebruiker,Geen gebruiker,Geen gebruiker
4697,4697,97,2.0,"Beloop: \tMR CPG7P4M4 // MI: 4x sectio ia, 1x...",Geen gebruiker,Geen gebruiker,Geen gebruiker
4698,4698,98,2.0,Reden van komst / Verwijzing: \tReden van koms...,Geen gebruiker,Geen gebruiker,Geen gebruiker


In [5]:
lines = list(fhlo['text'])

In [6]:
full_to_translate = []
for i, line in enumerate(lines):
    separator = "."
    new_line = line.replace("\t", separator)
    if len(new_line) > 512:
        last_index = new_line[:512].rfind(".") + 1
        if last_index < 1:
            last_index = 512
        lines_to_translate = [new_line[:last_index]]
        current_string = new_line[last_index:]
        while len(current_string) > 512:
            last_index = current_string[:512].rfind(".") + 1
            if last_index < 1:
                last_index = 512
            lines_to_translate.append(current_string[:last_index])
            current_string = current_string[last_index:]
        if len(current_string) > 0:
            lines_to_translate.append(current_string)
        full_to_translate.append(lines_to_translate)
        del(lines_to_translate)
        continue
    full_to_translate.append(new_line)

In [22]:
results = []
for i, item in enumerate(full_to_translate):
    if isinstance(item, str):
        results.append([i, 1, item])
        continue
    for j, subitem in enumerate(item):
        results.append([i, j, subitem])
    

In [23]:
result_df = pd.DataFrame(results)

In [24]:
result_df

,0,1,2
0,0,1,Beloop: .PatiÃ«nte heeft 10 minuten van te vor...
1,1,1,Beloop: .G5P4 36 wkGrav 1e lijnALL geen Hb Pat...
2,2,1,Conclusie: .Normale nacontrole. Kijkt goed ter...
3,3,1,Beleid: .Algemeen: Dagopname voor 3x PC a 2.5...
4,4,1,Anamnese: .34W4D
...,...,...,...
17069,4699,5,"Hoofd/hals: vochtige slijmvliezen, CVD niet ve..."
17070,4699,6,"Sinusritme 77/min, IMHA, normale geleidingstij..."
17071,4699,7,"Geen wandbewegingsstoornissen. Slanke VC, col..."
17072,4699,8,Near-collaps bij 1. .3. Pre-existente microcy...


In [25]:
result_df = result_df.rename(columns={0: 'doc_id', 1:'segment_id', 2:'text'})

In [26]:
from datasets import Dataset

dataset = Dataset.from_pandas(result_df)

In [27]:
dataset

Dataset({
    features: ['doc_id', 'segment_id', 'text'],
    num_rows: 17074
})

In [28]:
def translate_dataset(example):
    example['translated'] = translator(example['text'])[0]['translation_text']
    return example

In [29]:
tokenizer = AutoTokenizer.from_pretrained("../../opus-mt-nl-en")

model = AutoModelForSeq2SeqLM.from_pretrained("../../opus-mt-nl-en")

In [30]:
from transformers import pipeline

translator = pipeline("translation", model=model, tokenizer=tokenizer, device = 0)

In [31]:
translated_dataset = dataset.map(translate_dataset)

Map:   0%|          | 0/17074 [00:00<?, ? examples/s]

KeyboardInterrupt: 